In [2]:
import pandas

In [13]:
question = pandas.read_csv('./data/question.csv')
answer = pandas.read_csv('./data/answer.csv')

len(question), len(answer)

(120000, 226266)

In [ ]:
question.head()

,question_id,content
0,65102009,头痛恶心肌肉痛关节痛颈部淋巴结疼痛怎么回事啊
1,44275784,我怀孕37周，肠子一直呼噜呼噜叫感觉像是在里面灌水，上厕所拉稀和喷水一样，一天上厕所5次，对...
2,42163349,男，67岁，2天前突然出现右小腿（类似抽筋症状），现出现右小腿消肿，有指压痕，无，请问可能是...
3,67935540,怀孕前两个月照了CT和X光，来两次月经后怀孕了，怀孕期间有轻微盆腔积水吃了两瓶杏香兔耳片。请...
4,33429289,阴囊湿冷阳痿早泻请问用点什么药


In [6]:
answer.head()

,ans_id,question_id,content
0,0,45619783,月经延迟十四天而且伴随恶心，头痛，乏力的现象，那么考虑怀孕的概率是非常大的，建议你去医院检查...
1,1,45619783,如果你的月经周期规律，有正常的性生活，未采取任何有效的避孕措施，此时的症状考虑有怀孕的可能。...
2,2,45619783,建议在性生活过后14天左右可以用怀孕试纸自我检测一下，一般怀孕试纸显示2条线的话是怀孕了的，...
3,3,26616465,头痛是临床上最为常见的临床症状之一，是人体对各种致痛因素所产生的主观感觉，属于疼痛的范畴。建...
4,4,26616465,头痛主要是由于头部的血管、神经、脑膜等对疼痛敏感的组织受到刺激引起的。由紧张、疲劳、饮酒等原...


## Rule:

* 一个答案对应一个问题，一个问题对应多个答案。
* 不同答案拆分到不同的QA中
* 超长的答案拆分到不同的QA中


In [20]:
# 找出最长的回答的长度.

max_len = 0
for i in range(len(answer)):
    if len(answer.iloc[i]['content']) > max_len:
        max_len = len(answer.iloc[i]['content'])

In [21]:
max_len

249

In [22]:
# 找出最长的问题的长度.

max_len = 0
for i in range(len(question)):
    if len(question.iloc[i]['content']) > max_len:
        max_len = len(question.iloc[i]['content'])

In [23]:
max_len

149

无需特殊处理，无需拆分的答案，直接写在output中即可。

## 制作QA对

{  
    "instruction": "现在你要扮演皇帝身边的女人--甄嬛",  
    "input":"你是谁？",  
    "output":"家父是大理寺少卿甄远道。"  
}  

In [32]:
instruction = "碰到以下症状，可能是什么病症？症状是什么引起的，应该怎么治疗？"

In [33]:
# input 是问题，但是问题和回答的映射存在answer中
output = answer["content"][0]

# input 的 question_id 是 answer["question_id"][0]
question_id = answer["question_id"][0]
input = question[question["question_id"] == question_id]["content"].values[0]

# 写入字典
qa = {}
qa["instruction"] = instruction
qa["input"] = input
qa["output"] = output

In [36]:
qa

{'instruction': '碰到以下症状，可能是什么病症？症状是什么引起的，应该怎么治疗？',
 'input': '停经十四天，会怀孕吗停经十四天，特别恶心已经停经十四天了，性生活频繁，出现恶心的症状，而且特别频繁严重，头疼，易疲惫，以前来月经都特别准，推迟不会超过三天，请问这怀孕概率大吗',
 'output': '月经延迟十四天而且伴随恶心，头痛，乏力的现象，那么考虑怀孕的概率是非常大的，建议你去医院检查血HCG或者B超确诊是否怀孕，如果怀孕了，那么怀孕的天数大概是44天如果打算保留这个孩子，从现在开始要戒烟酒，不要同房，注意休息，饮食方面要少吃辛辣刺激的，腌制，油炸，烧烤的食物，不要吃凉性的食物，不要吃螃蟹甲鱼等寒性食品，容易流产，定期孕检'}

In [46]:
# 生成所有的 QA 对, 保存dict到 ./data/qa.json
from tqdm import tqdm
qa_list = []
for i in tqdm(range(len(answer))):
    instruction = "碰到以下症状，可能是什么病症？症状是什么引起的，应该怎么治疗？"
    output = answer["content"][i]
    question_id = answer["question_id"][i]
    input = question[question["question_id"] == question_id]["content"].values[0]
    qa = {}
    qa["instruction"] = instruction
    qa["input"] = input
    qa["output"] = output
    qa_list.append(qa)

  0%|          | 0/226266 [00:00<?, ?it/s]

100%|██████████| 226266/226266 [00:47<00:00, 4801.23it/s]


In [49]:
len(qa_list)

226266

In [47]:
qa_list

[{'instruction': '碰到以下症状，可能是什么病症？症状是什么引起的，应该怎么治疗？',
  'input': '停经十四天，会怀孕吗停经十四天，特别恶心已经停经十四天了，性生活频繁，出现恶心的症状，而且特别频繁严重，头疼，易疲惫，以前来月经都特别准，推迟不会超过三天，请问这怀孕概率大吗',
  'output': '月经延迟十四天而且伴随恶心，头痛，乏力的现象，那么考虑怀孕的概率是非常大的，建议你去医院检查血HCG或者B超确诊是否怀孕，如果怀孕了，那么怀孕的天数大概是44天如果打算保留这个孩子，从现在开始要戒烟酒，不要同房，注意休息，饮食方面要少吃辛辣刺激的，腌制，油炸，烧烤的食物，不要吃凉性的食物，不要吃螃蟹甲鱼等寒性食品，容易流产，定期孕检'},
 {'instruction': '碰到以下症状，可能是什么病症？症状是什么引起的，应该怎么治疗？',
  'input': '停经十四天，会怀孕吗停经十四天，特别恶心已经停经十四天了，性生活频繁，出现恶心的症状，而且特别频繁严重，头疼，易疲惫，以前来月经都特别准，推迟不会超过三天，请问这怀孕概率大吗',
  'output': '如果你的月经周期规律，有正常的性生活，未采取任何有效的避孕措施，此时的症状考虑有怀孕的可能。你的情况，如果担心怀孕，可严密观察，如果月经延迟一周以上，则到正规医院妇科进行血HCG、B超等相关检查，确诊是否怀孕'},
 {'instruction': '碰到以下症状，可能是什么病症？症状是什么引起的，应该怎么治疗？',
  'input': '停经十四天，会怀孕吗停经十四天，特别恶心已经停经十四天了，性生活频繁，出现恶心的症状，而且特别频繁严重，头疼，易疲惫，以前来月经都特别准，推迟不会超过三天，请问这怀孕概率大吗',
  'output': '建议在性生活过后14天左右可以用怀孕试纸自我检测一下，一般怀孕试纸显示2条线的话是怀孕了的，一条线的话没有怀孕，有2条线，一条线明显，另一条线不明显，那么是弱阳性，试纸显示弱阳性有两种可能，一种是怀孕了，一种是炎症。同样可以去医院做血HCG的检查，这个准确率是很高的。'},
 {'instruction': '碰到以下症状，可能是什么病症？症状是什么引起的，应该怎么治疗？',
  'input': '头痛头昏十几年前。吹风和晒太阳。吵闹

In [48]:
# 存储到文件
import json
with open('./data/qa.json', 'w') as f:
    json.dump(qa_list, f)

## 定义数据集格式化
Lora 训练的数据是需要经过格式化、编码之后再输入给模型进行训练的，如果是熟悉 Pytorch 模型训练流程的同学会知道，我们一般需要将输入文本编码为 input_ids，将输出文本编码为 labels，编码之后的结果都是多维的向量。我们首先定义一个预处理函数，这个函数用于对每一个样本，编码其输入、输出文本并返回一个编码后的字典： 

In [52]:
# 导入必要的库
from transformers import AutoTokenizer, PreTrainedTokenizer

tokenizer = AutoTokenizer.from_pretrained('./senmen-chat', use_fast=False, trust_remote_code=True)
tokenizer.pad_token_id = tokenizer.eod_id # Qwen中eod_id和pad_token_id是一样的，但需要指定一下

from typing import Tuple, List, Union, Iterable

HistoryType = List[Tuple[str, str]]
TokensType = List[int]
BatchTokensType = List[List[int]]

def make_context(
    tokenizer: PreTrainedTokenizer,
    query: str,
    history: List[Tuple[str, str]] = None,
    system: str = "",
    max_window_size: int = 6144,
    chat_format: str = "chatml",
):
    if history is None:
        history = []

    if chat_format == "chatml":
        im_start, im_end = "<|im_start|>", "<|im_end|>"
        im_start_tokens = [tokenizer.im_start_id]
        im_end_tokens = [tokenizer.im_end_id]
        nl_tokens = tokenizer.encode("\n")

        def _tokenize_str(role, content):
            return f"{role}\n{content}", tokenizer.encode(
                role, allowed_special=set()
            ) + nl_tokens + tokenizer.encode(content, allowed_special=set())

        system_text, system_tokens_part = _tokenize_str("system", system)
        system_tokens = im_start_tokens + system_tokens_part + im_end_tokens

        raw_text = ""
        context_tokens = []

        for turn_query, turn_response in reversed(history):
            query_text, query_tokens_part = _tokenize_str("user", turn_query)
            query_tokens = im_start_tokens + query_tokens_part + im_end_tokens
            response_text, response_tokens_part = _tokenize_str(
                "assistant", turn_response
            )
            response_tokens = im_start_tokens + response_tokens_part + im_end_tokens

            next_context_tokens = nl_tokens + query_tokens + nl_tokens + response_tokens
            prev_chat = (
                f"\n{im_start}{query_text}{im_end}\n{im_start}{response_text}{im_end}"
            )

            current_context_size = (
                len(system_tokens) + len(next_context_tokens) + len(context_tokens)
            )
            if current_context_size < max_window_size:
                context_tokens = next_context_tokens + context_tokens
                raw_text = prev_chat + raw_text
            else:
                break

        context_tokens = system_tokens + context_tokens
        raw_text = f"{im_start}{system_text}{im_end}" + raw_text
        context_tokens += (
            nl_tokens
            + im_start_tokens
            + _tokenize_str("user", query)[1]
            + im_end_tokens
            + nl_tokens
            + im_start_tokens
            + tokenizer.encode("assistant")
            + nl_tokens
        )
        raw_text += f"\n{im_start}user\n{query}{im_end}\n{im_start}assistant\n"

    elif chat_format == "raw":
        raw_text = query
        context_tokens = tokenizer.encode(raw_text)
    else:
        raise NotImplementedError(f"Unknown chat format {chat_format!r}")

    return raw_text, context_tokens

ValueError: Unrecognized model in ./senmen-chat. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: albert, align, altclip, audio-spectrogram-transformer, autoformer, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, blenderbot, blenderbot-small, blip, blip-2, bloom, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, conditional_detr, convbert, convnext, convnextv2, cpmant, ctrl, cvt, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deformable_detr, deit, depth_anything, deta, detr, dinat, dinov2, distilbert, donut-swin, dpr, dpt, efficientformer, efficientnet, electra, encodec, encoder-decoder, ernie, ernie_m, esm, falcon, falcon_mamba, fastspeech2_conformer, flaubert, flava, fnet, focalnet, fsmt, funnel, fuyu, gemma, gem

经过格式化的数据，也就是送入模型的每一条数据，都是一个字典，包含了 input_ids、attention_mask、labels 三个键值对，其中 input_ids 是输入文本的编码，attention_mask 是输入文本的 attention mask，labels 是输出文本的编码。  
经过decode之后应该还原出来原来的数据类似这样:  

```txt
<|im_start|>system
现在你要扮演皇帝身边的女人--甄嬛.<|im_end|>
<|im_start|>user
小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——<|im_end|>
<|im_start|>assistant
嘘——都说许愿说破是不灵的。<|im_end|>
<|endoftext|>
```